In [1]:
# Generate examples of 2 types 
# Very general and useful code
# Run model 

# Please read the source code 
# I put a lot of effort in it 

# Uncomment to get new dataset
# Taked 10 seconds
# !python generater.py ../configs/simple_sampler.yaml

In [2]:
import pandas as pd 
from ipywidgets import widgets 

In [3]:
df = pd.read_parquet('../data/prepared.parquet')
df.head()

,filename,prefix,middle,suffix,meta,query,output
0,../data/SDF-JEPA-main/evals/main.py,",\n default='configs.yaml')\nparser.add_arg...",)\n\n,import logging\n logging.basicConfig()\...,finish_lines,"<fim_prefix>,\n default='configs.yaml')\npa...",)\n os.environ['MASTER_ADDR'] = 'localhost'...
1,../data/SDF-JEPA-main/src/models/vision_transf...,"cale(layer.mlp.fc2.weight.data, layer_id + 1)\...","(remove)\n """"""\n\n",if masks is not None and not isinstanc...,finish_lines,"<fim_prefix>cale(layer.mlp.fc2.weight.data, la...","and mask tokens\n :return:\n """"""..."
2,../data/SDF-JEPA-main/app/vjepa/train.py,"th=dataset_eval_paths,\n batch_size=ba...",",\n transform=transform,\n","datasets_weights=datasets_weights,\n ...",finish_lines,"<fim_prefix>th=dataset_eval_paths,\n b...","_per_video,\n num_clips_per_clip=num_c..."
3,../data/SDF-JEPA-main/src/models/utils/patch_e...,"roj = nn.Conv2d(in_chans, embed_dim, kernel_si...","(\n self,\n patch_size=16,\n","tubelet_size=2,\n in_chans=3,\n...",finish_lines,"<fim_prefix>roj = nn.Conv2d(in_chans, embed_di...","(self, in_chans, embed_dim):\n super()...."
4,../data/SDF-JEPA-main/src/models/utils/multima...,outs = []\n for m in masks:\n ...,):\n if type(ctxt) is not list:\n ...,if type(tgt) is not list:\n ...,finish_lines,<fim_prefix> outs = []\n for m in mask...,"):\n ctxt = [self.backbone(x, masks=m) ..."


In [4]:
from IPython.display import HTML
import ipywidgets as widgets

def format_text(prefix, middle, suffix):
    # Replace \n with <br> for HTML line breaks
    prefix = prefix.replace('\n', '<br>')
    middle = middle.replace('\n', '<br>')
    suffix = suffix.replace('\n', '<br>')

    # Replace \t with a series of non-breaking spaces for HTML tabs
    prefix = prefix.replace('\t', '&nbsp;' * 4)
    middle = middle.replace('\t', '&nbsp;' * 4)
    suffix = suffix.replace('\t', '&nbsp;' * 4)

    return f"<pre>{prefix}<span style='color:red'>{middle}</span>{suffix}</pre>"

def create_widget(prefix, middle, suffix, output):

    output_true = widgets.Output()
    with output_true:
        display(HTML(format_text(prefix, middle, suffix)))

    output_pred = widgets.Output()
    with output_pred:
        display(HTML(format_text(prefix, output, suffix)))

    # Create titles
    title_true = widgets.HTML("<h3>True</h3>")
    title_pred = widgets.HTML("<h3>Predicted</h3>")

    # Stack titles and outputs vertically
    column_true = widgets.VBox([title_true, output_true])
    column_pred = widgets.VBox([title_pred, output_pred])

    # Add space between columns
    column_true.layout.margin = '0 20px 0 0'  # 20px right margin for the first column

    two_columns = widgets.HBox([column_true, column_pred])

    return two_columns

Let's look 5 random examples from both strategies 

## Finish Lines
Finish lines strategy tries to predict 1-2 lines,  
when user finished typing specific token.  
This could be predicting after "," or "." or ")"  
For the full list and more precise behaviour please read the config at source code

In [5]:
finish_lines_df = df.query('meta == "finish_lines"').sample(5, random_state=123).reset_index()

In [6]:
widget = create_widget(*finish_lines_df.loc[0, ['prefix', 'middle', 'suffix', 'output']])
display(widget)

## Good: 
- predicted argv correcly 
- started loop correctly

## Bad 
- Didn't take into account that i is compared to len, thus should start enumerate from 1
- Created random function 

## Overall subjective score
3 / 10

In [7]:
widget = create_widget(*finish_lines_df.loc[1, ['prefix', 'middle', 'suffix', 'output']])
display(widget)

Unfortunetly not great example, because it's just a comma  
## Good 
- Comma 
## Bad 
- Broken grammar
- Stupid repeated params

## Overall subjective score
1 / 10

In [8]:
widget = create_widget(*finish_lines_df.loc[2, ['prefix', 'middle', 'suffix', 'output']])
display(widget)

## Good 
- Correcly finished method header 

## Bad 
- Did nonesence to passed params 

## Overall subjective score 
2 / 10

In [9]:
widget = create_widget(*finish_lines_df.loc[3, ['prefix', 'middle', 'suffix', 'output']])
display(widget)

## Good 
- Looks plausable 
- Almost correct

## Bad 
- Wrong and unnecessary second argument 

## Overall subjective score 
4 / 10

In [10]:
widget = create_widget(*finish_lines_df.loc[4, ['prefix', 'middle', 'suffix', 'output']])
display(widget)

## Good
- Correctly created argument

## Bad 
- Not what intended 
- Wrong second arg

## Overall subjective score 
3 / 10

# Finish Words

Finish words strategy tries to predict a few words after user started typing one word. 

In [11]:
finish_words_df = df.query('meta == "finish_words"').sample(5, random_state=123).reset_index()

In [12]:
widget = create_widget(*finish_words_df.loc[0, ['prefix', 'middle', 'suffix', 'output']])
display(widget)

## Good 
- Basically correct grammar 

## Bad 
- Did nonesence 
  
## Overall subjective score 
2 / 10

In [13]:
widget = create_widget(*finish_words_df.loc[1, ['prefix', 'middle', 'suffix', 'output']])
display(widget)

## Good
- Nothing 

## Bad 
- Wrong args 
- Wrong grammar  

## Overall subjective score 
2 / 10

In [14]:
widget = create_widget(*finish_words_df.loc[2, ['prefix', 'middle', 'suffix', 'output']])
display(widget)

## Good 
- Correct

## Bad 
- Slightly more than required 

## Overall subjective score 
9 / 10

In [16]:
widget = create_widget(*finish_words_df.loc[3, ['prefix', 'middle', 'suffix', 'output']])
display(widget)

## Good 
- Plausable continuation
- Almost correcrt grammar 

## Bad 
- Not correct 

## Overall subjective score 
6 / 10 

In [17]:
widget = create_widget(*finish_words_df.loc[4, ['prefix', 'middle', 'suffix', 'output']])
display(widget)

## Good 
- Almost plausable 

## Bad 
- Not correct 

## Overall subjective score
4 / 10

# Hypothesis 

Predicting after a word was started is much better, than after a comma or any other separator 

In [19]:
from codebleu import calc_codebleu
from nltk.translate.bleu_score import sentence_bleu
import plotly.express as px

In [30]:
calc_codebleu(df.loc[:, 'middle'].to_list(), df.loc[:, 'output'].to_list(), lang='python')

TypeError: an integer is required